In [ ]:
import os
from options.test_options import TestOptions
from data import create_dataset
from models import create_model
import matplotlib.pyplot as plt
import argparse
import imageio
from pathlib import Path
import numpy as np
import h5py

In [ ]:
!python test.py --dataroot ../outputs/hdf5/img_2_img_full_test.h5 --name tv_inv_proof --model pix2pix --norm instance --direction BtoA --dataset_mode hdf5 --gpu_ids -1 --input_nc 1 --output_nc 1 --no_dropout --num_test 2802

In [ ]:
class TestOptions:
    def __init__(self):
        self.phase = 'test'
        self.dataroot = '../outputs/hdf5/img_2_img_full_tv.h5'
        self.input_nc = 1
        self.output_nc = 1
        self.direction = 'BtoA'
        self.dataset_mode = 'hdf5_test'
        self.batch_size = 1
        self.num_threads = 0
        self.max_dataset_size = float("inf")
        self.model = 'pix2pix'
        self.gpu_ids = [0]
        self.isTrain = False
        self.no_dropout = True
        self.display_id = -1
        self.num_threads = 0
        self.batch_size = 1
        self.eval = True
        self.checkpoints_dir = './checkpoints'
        self.name = 'tv_inv_proof'
        self.preprocess = 'none'
        self.ngf = 64
        self.netD = 'basic'
        self.netG = 'resnet_9blocks'
        self.norm = 'instance'
        self.init_type = "normal"
        self.init_gain = 0.02
        self.load_iter = 0
        self.epoch = 'latest'
        
        
opt = TestOptions()

In [ ]:
print(len(dataset))

In [ ]:
c = 200
d = 0
for data in dataset:
    d += 1
    a = data['A'][0][0]
    if d == c:
        break

In [ ]:
plt.imshow(a)

In [ ]:
!dir

In [ ]:
import cv2
image_dir = Path("results/tv_inv_proof/test_latest/images/")

# Get all the image files that end in "fake_B"
image_files = list(image_dir.glob("*fake_B.png"))

# Create a list to store the images
images = []

# Read and append each image to the list
for image_file in image_files:
    image = plt.imread(image_file)
    images.append(image)

In [ ]:
h5_path = Path("../outputs/hdf5/img_2_img_full_test.h5")
with h5py.File(h5_path, "r") as f:
    a = f["A_test"][:]
a = [np.flip(image, axis=0) for image in a]

In [ ]:
# Convert images in a to black and white
bw_images = []
for image in images:
    bw_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    bw_images.append(bw_image)
bw_images = [np.flip(image, axis=0) for image in bw_images]


In [ ]:
print(bw_images[0].shape)

In [ ]:
import matplotlib.gridspec as gridspec
import matplotlib.animation as animation

In [ ]:
# Create the figure and axes
fig = plt.figure()
gs0 = gridspec.GridSpec(1,2, figure=fig)

ax1 = fig.add_subplot(gs0[0])
ax2 = fig.add_subplot(gs0[1])

img1 = ax1.imshow(bw_images[0], cmap='plasma')
img2 = ax2.imshow(a[0], cmap='plasma')
ax1.axis("off")
ax2.axis("off")
fig.subplots_adjust(
    left=0, bottom=0.0, right=1, top=1, wspace=None, hspace=None
)

# # Define the update function for the animation
def update(frame):
    img1.set_array(bw_images[frame])
    img2.set_array(a[frame])
    return ax1, ax2

# # Create the animation
ani = animation.FuncAnimation(
    fig, update, frames=range(len(a)), interval=30, blit=False
)

# Save the animation to a file
output_file = Path("ok").with_suffix(".mp4")

FFwriter = animation.FFMpegWriter(fps=30)
ani.save(output_file, writer=FFwriter)